# Analysis

In this section, the avaibale dataset used in this project are presented.
* __Hint__   
To have a better view of this notebook, click here:  
https://nbviewer.jupyter.org/github/haamedmomeni/github-example/blob/master/DataAnalysis.ipynb  
Thanks!

## NYS population distribution in counties

In [2]:
import pandas as pd
import numpy as np
url = 'https://wikipedia.org/wiki/List_of_counties_in_New_York'
df = pd.read_html(url)[2]
df.head()

,County,FIPS Code[3],County seat[5],Est.[5],Formed from[1],Named for[2],Density (Pop./mi2),Pop. (2010)[6],Area[5],Map
0,Albany County,1,Albany,1683,One of 12 original counties created in the New...,James II of England (James VII of Scotland) (1...,570.74,304204,"533 sq mi(1,380 km2)",NaN
1,Allegany County,3,Belmont,1806,Genesee County,A variant spelling of the Allegheny River,47.34,48946,"1,034 sq mi(2,678 km2)",NaN
2,Bronx County,5,none,1914[7],New York County,"Jonas Bronck (1600?–1643), an early settler of...",24118.20,1385108,57.43 sq mi(149 km2),NaN
3,Broome County,7,Binghamton,1806,Tioga County,"John Broome (1738–1810), fourth Lieutenant Gov...",280.56,200600,"715 sq mi(1,852 km2)",NaN
4,Cattaraugus County,9,Little Valley,1808,Genesee County,A word from an uncertain Iroquoian language me...,61.31,80317,"1,310 sq mi(3,393 km2)",NaN


### Remove outlier columns

In [3]:
df1 = pd.DataFrame()
df1 = df[["County","Pop. (2010)[6]","Density (Pop./mi2)"]].copy()
df1 = df1.rename(columns={"Pop. (2010)[6]": "Population"})
df1['Area (mi2)'] = df1['Population']/df1['Density (Pop./mi2)']
df1['Area (mi2)'] = df1['Area (mi2)'].round(2)
df1.head()

,County,Population,Density (Pop./mi2),Area (mi2)
0,Albany County,304204,570.74,533.00
1,Allegany County,48946,47.34,1033.92
2,Bronx County,1385108,24118.20,57.43
3,Broome County,200600,280.56,715.00
4,Cattaraugus County,80317,61.31,1310.01


## Location of each county

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

AddresLat = list()
AddresLong = list()


for county in df1['County']:
    address =  county + ', NY'
    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    AddresLat.append(latitude)
    AddresLong.append(longitude)

In [5]:
df1['Latitude'] =  AddresLat
df1['Longitude'] =  AddresLong
df1.head()

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude
0,Albany County,304204,570.74,533.00,42.598690,-73.984400
1,Allegany County,48946,47.34,1033.92,42.244606,-78.041928
2,Bronx County,1385108,24118.20,57.43,40.850485,-73.840404
3,Broome County,200600,280.56,715.00,42.145562,-75.840411
4,Cattaraugus County,80317,61.31,1310.01,42.223482,-78.647710


## Average land price in each county

In [6]:
url = 'https://www.tax.ny.gov/research/property/assess/sales/resmedian.htm'
df = pd.read_html(url)[0]
df.head()

Unnamed: 0_level_0    2017              2018              2019          
              County # Sales    Median # Sales    Median # Sales    Median
0             Albany    3403  $207,000    3434  $210,000    3445  $217,500
1           Allegany     523   $61,500     509   $67,000     482   $75,500
2             Broome    1536  $110,000    1584  $116,000    1746  $110,000
3        Cattaraugus     804   $79,500     811   $80,000     843   $87,250
4             Cayuga     743  $117,000     781  $122,000     786  $120,250

### Remove outlier columns

In [7]:
CountyData = df.iloc[:, 0]
PriceData = df.iloc[:, 6]
df2 = pd.DataFrame()
df2['County'] =  CountyData
df2['LandPrice'] =  PriceData
df2['LandPrice'] = df2['LandPrice'].str.replace(',', '')
df2['LandPrice'] = df2['LandPrice'].str.replace('$', '')
df2['LandPrice'] = df2['LandPrice'].astype(int)
df2.head()

,County,LandPrice
0,Albany,217500
1,Allegany,75500
2,Broome,110000
3,Cattaraugus,87250
4,Cayuga,120250


## Merge two dataframes

In [8]:
df1['County'] = df1['County'].str.replace(' County', '')
dfinal = df1
dfinal = dfinal.merge(df2,how='left', left_on='County', right_on='County')
dfinal.head()

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude,LandPrice
0,Albany,304204,570.74,533.00,42.598690,-73.984400,217500.0
1,Allegany,48946,47.34,1033.92,42.244606,-78.041928,75500.0
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404,NaN
3,Broome,200600,280.56,715.00,42.145562,-75.840411,110000.0
4,Cattaraugus,80317,61.31,1310.01,42.223482,-78.647710,87250.0


In [9]:
latitude = 43.015598
longitude = -76.225713
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7) # generate map centred around the Conrad Hotel


# add the Italian restaurants as blue circle markers
for lat, lng, label, rad in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, dfinal.Population):
    folium.features.CircleMarker(
        [lat, lng],
        radius=rad/200000,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

## Add average prices to missed values for prices

In [10]:
missing_data = dfinal.isnull()
missing_data.head()
avg_price = dfinal.LandPrice.astype("float").mean(axis=0)
print("Average of price:", avg_price)
dfinal.LandPrice.replace(np.nan, avg_price, inplace=True)

Average of price: 179258.07142857142


In [11]:
latitude = 43.015598
longitude = -76.225713
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7) # generate map centred around the Conrad Hotel

# add the Italian restaurants as blue circle markers
for lat, lng, label, rad in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, dfinal.LandPrice):
    folium.features.CircleMarker(
        [lat, lng],
        radius=rad/20000,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

To see the map use this link!  
https://nbviewer.jupyter.org/github/haamedmomeni/github-example/blob/master/Data.ipynb

## Clustering
A primary clustering will do to have an estimation of diffences between counties.

In [135]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
x = dfinal.values[:,1:]
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset
num_clusters = 4

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=50)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 3 2 0 3 0 3 0 0 3 0 0 3 0 3 1 1 0 0 0 1 3 1 2 3 0 0 3 0 0 2 3 3 0 0 0 0
 3 3 0 0 0 0 0 1 0 0 0 0 0 3 1 3 0 0 3 0 0 3 0 0 0]


In [136]:
df1["Labels"] = labels
df1.head(5)

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude,Labels,WeightedDistance
0,Albany,304204,570.74,533.00,42.598690,-73.984400,0,4.047503e+07
1,Allegany,48946,47.34,1033.92,42.244606,-78.041928,3,7.303513e+07
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404,2,2.675696e+07
3,Broome,200600,280.56,715.00,42.145562,-75.840411,0,4.474017e+07
4,Cattaraugus,80317,61.31,1310.01,42.223482,-78.647710,3,8.265323e+07


In [137]:
latitude = 42.8
longitude = -76.2
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7)

import matplotlib.cm as cm
import matplotlib.colors as colors
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add the Italian restaurants as blue circle markers
for lat, lng, label, cluster in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, df1.Labels):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=rainbow[cluster-1],
        popup=label,
        fill = True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

In [138]:
WeightedDistance = list()

for i in range(0,62):
    dist = 0
    for j in range(0,62):
        dist = dist + ( (df1.iloc[i, 5] - df1.iloc[j, 5])**2 + (df1.iloc[i, 4] - df1.iloc[j, 4])**2 )**0.5 * df1.iloc[j, 1]
    WeightedDistance.append(dist)

df1['WeightedDistance'] =  WeightedDistance
df1.head()

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude,Labels,WeightedDistance
0,Albany,304204,570.74,533.00,42.598690,-73.984400,0,4.047503e+07
1,Allegany,48946,47.34,1033.92,42.244606,-78.041928,3,7.303513e+07
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404,2,2.675696e+07
3,Broome,200600,280.56,715.00,42.145562,-75.840411,0,4.474017e+07
4,Cattaraugus,80317,61.31,1310.01,42.223482,-78.647710,3,8.265323e+07


In [152]:
latitude = 42.8
longitude = -76.2
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7)

for lat, lng, label, rad in zip(dfinal.Latitude, dfinal.Longitude, dfinal.County, df1.WeightedDistance):
    folium.features.CircleMarker(
        [lat, lng],
        radius=200000000/rad,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

In [182]:
Cost = list()

for i in range(0,62):
    cost = 0
    for j in range(0,62):
        cost = cost + df1.iloc[i, 7]  + 200 * dfinal.iloc[i, 6]
    Cost.append(cost)

df1['Cost'] =  Cost

In [184]:
df7 = df1.sort_values(by='Cost', ascending=True)
df7.head(5)

,County,Population,Density (Pop./mi2),Area (mi2),Latitude,Longitude,Labels,WeightedDistance,Cost
52,Sullivan,77547,77.78,997.00,41.715631,-74.780432,3,3.400705e+07,3.813437e+09
30,New York,1585873,46961.00,33.77,40.781018,-73.959300,2,2.671660e+07,3.879229e+09
2,Bronx,1385108,24118.20,57.43,40.850485,-73.840404,2,2.675696e+07,3.881732e+09
28,Montgomery,50219,122.49,409.98,42.894127,-74.409975,0,4.362945e+07,3.883026e+09
8,Chenango,50477,56.16,898.81,42.478457,-75.613028,0,4.469691e+07,3.887209e+09


In [190]:
latitude = 41.715631
longitude = -74.780432
Countis_map = folium.Map(location=[latitude, longitude], zoom_start=7)
folium.features.CircleMarker(
        [latitude, longitude],
        radius=25,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Countis_map )

# display map
Countis_map 

* __Hint__   
To have a better view of this notebook, click here:  
https://nbviewer.jupyter.org/github/haamedmomeni/github-example/blob/master/DataAnalysis.ipynb  
Thanks!